# Carregamento de Dados Abertos do Gov.br

Este notebook demonstra como carregar dados de diversas fontes governamentais brasileiras:
- Portal Brasileiro de Dados Abertos (dados.gov.br)
- Portal da Transparência
- Ministério da Saúde
- Tesouro Nacional

## 1. Instalação de bibliotecas necessárias

In [ ]:
!pip install requests pandas minio

## 2. Importar bibliotecas

In [ ]:
import requests
import pandas as pd
import json
from datetime import datetime
from minio import Minio
from minio.error import S3Error
import io

## 3. Configurar MinIO para salvar dados

In [ ]:
# Configurações do MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

# API Key do Portal da Transparência
PORTAL_TRANSPARENCIA_API_KEY = "2c56919ba91b8c1b13473dcef43fb031"

# Criar cliente MinIO
minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

# Verificar/criar bucket
if not minio_client.bucket_exists(BUCKET_NAME):
    minio_client.make_bucket(BUCKET_NAME)
    print(f"Bucket '{BUCKET_NAME}' criado")
else:
    print(f"Bucket '{BUCKET_NAME}' já existe")
    
print(f"API Key Portal da Transparência configurada!")

## 4. Função auxiliar para salvar no MinIO

In [ ]:
def save_to_minio(df, object_name, file_format='csv'):
    """Salva DataFrame no MinIO"""
    try:
        # Converter DataFrame para bytes
        if file_format == 'csv':
            data = df.to_csv(index=False).encode('utf-8')
            content_type = 'text/csv'
        elif file_format == 'json':
            data = df.to_json(orient='records', force_ascii=False).encode('utf-8')
            content_type = 'application/json'
        elif file_format == 'parquet':
            buffer = io.BytesIO()
            df.to_parquet(buffer, index=False)
            data = buffer.getvalue()
            content_type = 'application/octet-stream'
        
        # Upload para MinIO
        minio_client.put_object(
            BUCKET_NAME,
            object_name,
            io.BytesIO(data),
            length=len(data),
            content_type=content_type
        )
        print(f"✓ Arquivo salvo: {object_name} ({len(data)} bytes)")
        return True
    except Exception as e:
        print(f"✗ Erro ao salvar: {e}")
        return False

## 5. Portal Brasileiro de Dados Abertos (dados.gov.br)

### 5.1. Listar datasets disponíveis

In [ ]:
# API do Portal de Dados Abertos
base_url = "https://dados.gov.br/api/3/action"

# Listar alguns datasets
response = requests.get(f"{base_url}/package_list")
if response.status_code == 200:
    data = response.json()
    datasets = data['result'][:10]  # Primeiros 10 datasets
    print("Primeiros 10 datasets disponíveis:")
    for i, dataset in enumerate(datasets, 1):
        print(f"{i}. {dataset}")
else:
    print(f"Erro: {response.status_code}")

### 5.2. Buscar datasets por tema

In [ ]:
# Buscar datasets sobre saúde
params = {
    'q': 'saúde',
    'rows': 5
}

response = requests.get(f"{base_url}/package_search", params=params)
if response.status_code == 200:
    data = response.json()
    print(f"Total de datasets encontrados: {data['result']['count']}\n")
    
    for pkg in data['result']['results']:
        print(f"Título: {pkg['title']}")
        print(f"Nome: {pkg['name']}")
        print(f"Organização: {pkg.get('organization', {}).get('title', 'N/A')}")
        print(f"Recursos: {len(pkg['resources'])}")
        print("-" * 80)
else:
    print(f"Erro: {response.status_code}")

### 5.3. Baixar um dataset específico

In [ ]:
# Exemplo: Baixar informações de um dataset
dataset_name = "bolsa-familia-pagamentos"  # Substitua pelo nome desejado

response = requests.get(f"{base_url}/package_show", params={'id': dataset_name})
if response.status_code == 200:
    data = response.json()
    pkg = data['result']
    
    print(f"Dataset: {pkg['title']}")
    print(f"Descrição: {pkg.get('notes', 'N/A')[:200]}...")
    print(f"\nRecursos disponíveis ({len(pkg['resources'])}):\n")
    
    for i, resource in enumerate(pkg['resources'][:5], 1):
        print(f"{i}. {resource['name']}")
        print(f"   Formato: {resource.get('format', 'N/A')}")
        print(f"   URL: {resource['url'][:80]}...")
        print()
else:
    print(f"Dataset '{dataset_name}' não encontrado")

## 6. Portal da Transparência

### 6.1. Consultar Bolsa Família

In [ ]:
# API Portal da Transparência (COM AUTENTICAÇÃO)
transparency_url = "http://api.portaldatransparencia.gov.br/api-de-dados"

# Headers com a chave de API
headers = {
    'chave-api-dados': PORTAL_TRANSPARENCIA_API_KEY
}

# IMPORTANTE: A API tem mudado os nomes dos programas
# Bolsa Família -> Auxílio Brasil (2021) -> Bolsa Família novamente (2023)
# Vamos tentar BPC (Benefício de Prestação Continuada) que funciona bem

print("Consultando BPC (Benefício de Prestação Continuada)...")

params = {
    'mesAno': '202412',  # Dezembro 2024
    'codigoIbge': '3550308',  # São Paulo
    'pagina': 1
}

try:
    response = requests.get(
        f"{transparency_url}/bpc-por-municipio",
        headers=headers,
        params=params,
        timeout=30
    )
    
    if response.status_code == 200:
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            print(f"Dados BPC - Dezembro 2024 - São Paulo:")
            print(f"Total de registros: {len(df)}")
            print(df.head())
            
            # Salvar no MinIO
            save_to_minio(df, "transparencia/bpc_202412_SP.csv")
        else:
            print("Nenhum dado retornado")
    else:
        print(f"Erro na API: {response.status_code}")
        print(f"Resposta: {response.text[:200]}")
except Exception as e:
    print(f"Erro: {e}")

### 6.2. Download de arquivos CSV do Portal da Transparência

In [ ]:
# Listar todos os órgãos do governo federal
print("Listando órgãos SIAFI...")

try:
    response = requests.get(
        f"{transparency_url}/orgaos-siafi",
        headers=headers,
        timeout=30
    )
    
    if response.status_code == 200:
        data = response.json()
        df_orgaos = pd.DataFrame(data)
        
        # Filtrar apenas órgãos válidos (não são CODIGO INVALIDO)
        df_orgaos = df_orgaos[~df_orgaos['descricao'].str.contains('CODIGO INVALIDO', na=False)]
        
        print(f"Total de órgãos: {len(df_orgaos)}")
        print(df_orgaos.head(20))
        
        # Salvar no MinIO
        save_to_minio(df_orgaos, "transparencia/orgaos_siafi.csv")
    else:
        print(f"Erro: {response.status_code}")
except Exception as e:
    print(f"Erro: {e}")

### 6.1c. Listar órgãos do governo (SIAFI)

In [ ]:
# Coletar BPC de todos os municípios de SP
print("Coletando dados de BPC para todos os municípios de SP...")

# Primeiro, pegar lista de municípios de SP do IBGE
ibge_url = "https://servicodados.ibge.gov.br/api/v1"
response_mun = requests.get(f"{ibge_url}/localidades/estados/35/municipios")

if response_mun.status_code == 200:
    municipios_sp = response_mun.json()
    print(f"Total de municípios em SP: {len(municipios_sp)}")
    
    # Coletar dados de BPC para os primeiros 10 municípios (para exemplo)
    bpc_data = []
    
    for i, municipio in enumerate(municipios_sp[:10], 1):
        codigo_ibge = str(municipio['id'])
        nome = municipio['nome']
        
        print(f"{i}/10 - Buscando {nome}...")
        
        try:
            response = requests.get(
                f"{transparency_url}/bpc-por-municipio",
                headers=headers,
                params={'mesAno': '202412', 'codigoIbge': codigo_ibge, 'pagina': 1},
                timeout=10
            )
            
            if response.status_code == 200:
                data = response.json()
                if data:
                    bpc_data.extend(data)
        except Exception as e:
            print(f"  Erro em {nome}: {e}")
    
    if bpc_data:
        df_bpc = pd.DataFrame(bpc_data)
        print(f"\nTotal de registros coletados: {len(df_bpc)}")
        print(df_bpc.head())
        
        # Salvar no MinIO
        save_to_minio(df_bpc, "transparencia/bpc_202412_SP_municipios.csv")
    else:
        print("Nenhum dado coletado")
else:
    print("Erro ao buscar municípios do IBGE")

### 6.1b. BPC - Todos os municípios de um estado

### 6.2. Consultar Despesas Públicas

In [ ]:
print("Endpoints disponíveis no Portal da Transparência:")
print("=" * 80)
print()

endpoints = {
    "Bolsa Família": {
        "endpoint": "/bolsa-familia-por-municipio",
        "params": "mesAno, codigoIbge, pagina"
    },
    "Bolsa Família por CPF": {
        "endpoint": "/bolsa-familia-por-cpf-ou-nis",
        "params": "cpfOuNis, mesAno, pagina"
    },
    "Seguro Defeso": {
        "endpoint": "/seguro-defeso",
        "params": "mesAno, codigoIbge, pagina"
    },
    "Garantia-Safra": {
        "endpoint": "/garantia-safra",
        "params": "mesAno, codigoIbge, pagina"
    },
    "Despesas Públicas": {
        "endpoint": "/despesas/documentos",
        "params": "mesAnoInicio, mesAnoFim, pagina"
    },
    "Servidores": {
        "endpoint": "/servidores",
        "params": "mesAnoReferencia, codigoOrgao, pagina"
    },
    "Convênios": {
        "endpoint": "/convenios",
        "params": "dataInicialCelebracao, dataFinalCelebracao, pagina"
    },
    "Contratos": {
        "endpoint": "/contratos",
        "params": "dataInicial, dataFinal, pagina"
    },
    "Licitações": {
        "endpoint": "/licitacoes",
        "params": "dataInicial, dataFinal, pagina"
    },
    "CEIS (Empresas Inidôneas)": {
        "endpoint": "/ceis",
        "params": "cpfOuCnpj, pagina"
    },
    "CNEP (Empresas Punidas)": {
        "endpoint": "/cnep",
        "params": "cpfOuCnpj, pagina"
    },
    "CEAF (Medicamentos)": {
        "endpoint": "/ceaf",
        "params": "mesAno, codigoIbge, pagina"
    }
}

for nome, info in endpoints.items():
    print(f"📊 {nome}")
    print(f"   Endpoint: {info['endpoint']}")
    print(f"   Parâmetros: {info['params']}")
    print()

print("=" * 80)
print("Documentação completa: https://portaldatransparencia.gov.br/api-de-dados")

### 6.5. Listar endpoints disponíveis na API Portal da Transparência

In [ ]:
# Consultar convênios celebrados
params = {
    'dataInicialCelebracao': '01/01/2024',
    'dataFinalCelebracao': '31/03/2024',
    'pagina': 1
}

try:
    response = requests.get(
        f"{transparency_url}/convenios",
        headers=headers,
        params=params,
        timeout=30
    )
    
    if response.status_code == 200:
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            print(f"Convênios (Jan-Mar 2024):")
            print(f"Total de registros: {len(df)}")
            print(df.head())
            
            # Salvar no MinIO
            save_to_minio(df, "transparencia/convenios_2024_Q1.csv")
        else:
            print("Nenhum dado retornado")
    else:
        print(f"Erro: {response.status_code}")
        print(f"Resposta: {response.text[:200]}")
except Exception as e:
    print(f"Erro: {e}")

### 6.4. Consultar Convênios

In [ ]:
# Consultar servidores por órgão
params = {
    'mesAnoReferencia': '202401',
    'codigoOrgao': '20101',  # Presidência da República
    'pagina': 1
}

try:
    response = requests.get(
        f"{transparency_url}/servidores",
        headers=headers,
        params=params,
        timeout=30
    )
    
    if response.status_code == 200:
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            print(f"Servidores Públicos - Jan 2024:")
            print(f"Total de registros: {len(df)}")
            print(df.head())
            print(f"\nColunas disponíveis: {df.columns.tolist()}")
            
            # Salvar no MinIO
            save_to_minio(df, "transparencia/servidores_202401.csv")
        else:
            print("Nenhum dado retornado")
    else:
        print(f"Erro: {response.status_code}")
        print(f"Resposta: {response.text[:200]}")
except Exception as e:
    print(f"Erro: {e}")

### 6.3. Consultar Servidores Públicos

In [ ]:
# Consultar despesas por órgão
# Parâmetros: ano, mês, código do órgão, página

params = {
    'mesAnoInicio': '202401',
    'mesAnoFim': '202403',
    'pagina': 1
}

try:
    response = requests.get(
        f"{transparency_url}/despesas/por-orgao",
        headers=headers,
        params=params,
        timeout=30
    )
    
    if response.status_code == 200:
        data = response.json()
        if data:
            df = pd.DataFrame(data)
            print(f"Despesas Públicas (Jan-Mar 2024):")
            print(f"Total de registros: {len(df)}")
            print(df.head())
            
            # Salvar no MinIO
            save_to_minio(df, "transparencia/despesas_202401_202403.csv")
        else:
            print("Nenhum dado retornado")
    else:
        print(f"Erro: {response.status_code}")
        print(f"Resposta: {response.text[:200]}")
except Exception as e:
    print(f"Erro: {e}")

In [ ]:
# Portal da Transparência disponibiliza arquivos para download direto
# URL: https://portaldatransparencia.gov.br/download-de-dados

# Exemplo: Servidores Civis do Executivo Federal
# Nota: Estes são arquivos grandes, ajuste conforme necessário

print("Para download de arquivos completos, acesse:")
print("https://portaldatransparencia.gov.br/download-de-dados")
print("\nCategorias disponíveis:")
print("- Bolsa Família")
print("- Servidores Públicos")
print("- Despesas Públicas")
print("- Convênios")
print("- Licitações")
print("- Contratos")
print("- Garantia-Safra")
print("- Seguro Defeso")

## 7. Dados do Ministério da Saúde

### 7.1. Dados COVID-19

In [ ]:
# API OpenDataSUS
# Dados de COVID-19 do Brasil

covid_url = "https://covid-api.mmediagroup.fr/v1"

try:
    # Dados do Brasil
    response = requests.get(f"{covid_url}/cases?country=Brazil")
    
    if response.status_code == 200:
        data = response.json()
        
        # Converter para DataFrame
        records = []
        for state, info in data.items():
            if isinstance(info, dict):
                record = {
                    'estado': state,
                    'confirmados': info.get('confirmed', 0),
                    'recuperados': info.get('recovered', 0),
                    'mortes': info.get('deaths', 0)
                }
                records.append(record)
        
        df = pd.DataFrame(records)
        print("Dados COVID-19 Brasil (por estado):")
        print(df.head(10))
        
        # Salvar no MinIO
        timestamp = datetime.now().strftime('%Y%m%d')
        save_to_minio(df, f"saude/covid19_estados_{timestamp}.csv")
    else:
        print(f"Erro: {response.status_code}")
except Exception as e:
    print(f"Erro ao buscar dados: {e}")

### 7.2. Dados do DATASUS (exemplo)

In [ ]:
# DATASUS - Sistema de Informações sobre Nascidos Vivos (SINASC)
# Dados disponíveis em: https://opendatasus.saude.gov.br/

print("Dados do DATASUS disponíveis em:")
print("https://opendatasus.saude.gov.br/")
print("\nSistemas disponíveis:")
print("- SINASC (Nascidos Vivos)")
print("- SIM (Mortalidade)")
print("- SINAN (Agravos de Notificação)")
print("- SIH (Internações Hospitalares)")
print("- SIA (Ambulatorial)")
print("- CNES (Cadastro Nacional de Estabelecimentos de Saúde)")

## 8. Tesouro Nacional

### 8.1. Dados da Dívida Pública

In [ ]:
# API Tesouro Transparente
tesouro_url = "https://apidatalake.tesouro.gov.br/ords/custos/tt"

try:
    # Exemplo: Dívida Pública Federal
    response = requests.get(f"{tesouro_url}/dpf", timeout=10)
    
    if response.status_code == 200:
        data = response.json()
        
        if 'items' in data:
            df = pd.DataFrame(data['items'])
            print("Dados da Dívida Pública Federal:")
            print(df.head())
            print(f"\nTotal de registros: {len(df)}")
            
            # Salvar no MinIO
            timestamp = datetime.now().strftime('%Y%m%d')
            save_to_minio(df, f"tesouro/divida_publica_{timestamp}.csv")
        else:
            print("Formato de resposta inesperado")
    else:
        print(f"Erro: {response.status_code}")
except Exception as e:
    print(f"Erro: {e}")

## 9. IBGE - Dados Estatísticos

### 9.1. Consultar municípios

In [ ]:
# API IBGE - Localidades
ibge_url = "https://servicodados.ibge.gov.br/api/v1"

# Listar todos os municípios
response = requests.get(f"{ibge_url}/localidades/municipios")

if response.status_code == 200:
    municipios = response.json()
    
    # Converter para DataFrame
    data = []
    for mun in municipios:
        data.append({
            'codigo_ibge': mun['id'],
            'municipio': mun['nome'],
            'uf': mun['microrregiao']['mesorregiao']['UF']['sigla'],
            'regiao': mun['microrregiao']['mesorregiao']['UF']['regiao']['nome']
        })
    
    df = pd.DataFrame(data)
    print(f"Total de municípios: {len(df)}")
    print("\nPrimeiros 10 municípios:")
    print(df.head(10))
    
    # Salvar no MinIO
    save_to_minio(df, "ibge/municipios_brasil.csv")
else:
    print(f"Erro: {response.status_code}")

### 9.2. População estimada por município

In [ ]:
# Consultar dados do Censo/estimativas
# Endpoint de indicadores IBGE

# Código do indicador de população
indicador = "29171"  # População estimada

response = requests.get(
    f"{ibge_url}/agregados/{indicador}/periodos/2021/variaveis/93?localidades=N6[all]"
)

if response.status_code == 200:
    data = response.json()
    print("Dados de população por município obtidos com sucesso")
    print(f"Estrutura da resposta: {data[0].keys() if data else 'vazia'}")
    
    # Processar os dados conforme necessário
    # (A estrutura pode variar dependendo do indicador)
else:
    print(f"Erro: {response.status_code}")

## 10. Função genérica para download e armazenamento

In [ ]:
def download_and_save(url, object_name, file_format='csv', params=None):
    """
    Baixa dados de uma URL e salva no MinIO
    
    Args:
        url: URL da API ou arquivo
        object_name: Nome do objeto no MinIO
        file_format: Formato do arquivo (csv, json, parquet)
        params: Parâmetros para a requisição
    """
    try:
        response = requests.get(url, params=params, timeout=30)
        
        if response.status_code == 200:
            # Se a resposta for JSON, converter para DataFrame
            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
                
                # Tentar diferentes estruturas de JSON
                if isinstance(data, list):
                    df = pd.DataFrame(data)
                elif isinstance(data, dict):
                    if 'items' in data:
                        df = pd.DataFrame(data['items'])
                    elif 'result' in data:
                        df = pd.DataFrame(data['result'])
                    else:
                        df = pd.DataFrame([data])
                else:
                    print("Formato JSON não reconhecido")
                    return False
                
                # Salvar no MinIO
                return save_to_minio(df, object_name, file_format)
            
            # Se for CSV direto
            elif url.endswith('.csv') or 'text/csv' in response.headers.get('content-type', ''):
                df = pd.read_csv(io.StringIO(response.text))
                return save_to_minio(df, object_name, file_format)
            
            else:
                print(f"Tipo de conteúdo não suportado: {response.headers.get('content-type')}")
                return False
        else:
            print(f"Erro HTTP: {response.status_code}")
            return False
            
    except Exception as e:
        print(f"Erro ao processar: {e}")
        return False

print("Função download_and_save() criada!")

## 11. Exemplo de uso da função genérica

In [ ]:
# Exemplo: Download de dados do IBGE
download_and_save(
    url="https://servicodados.ibge.gov.br/api/v1/localidades/estados",
    object_name="ibge/estados_brasil.csv",
    file_format='csv'
)

## 12. Listar arquivos salvos no MinIO

In [ ]:
try:
    objects = minio_client.list_objects(BUCKET_NAME, recursive=True)
    print(f"Arquivos salvos no bucket '{BUCKET_NAME}':")
    print("=" * 80)
    
    total_size = 0
    count = 0
    
    for obj in objects:
        size_kb = obj.size / 1024
        total_size += obj.size
        count += 1
        print(f"📄 {obj.object_name}")
        print(f"   Tamanho: {size_kb:.2f} KB")
        print(f"   Modificado: {obj.last_modified}")
        print()
    
    if count == 0:
        print("Nenhum arquivo encontrado")
    else:
        print("=" * 80)
        print(f"Total: {count} arquivos ({total_size/1024/1024:.2f} MB)")
        
except S3Error as e:
    print(f"Erro ao listar objetos: {e}")